En el archivo .log están:

#energía total: "Total KS-DFT energy"
#cargas Mulliken por orbital por atomo: "Mulliken charges per centre and basis function type"
cargas de Loprop por atomo: "  LoProp Charges per center "

Dentro de "Molecular properties"
#Dipolo molec:   " Dipole Moment (Debye) "
quadrupolo molec: " Quadrupole Moment "

Dentro de cada DOMAIN
propiedades localizadas: (para átomos y para enlaces)
#carga "Total charge   "
dipolo electronico "Electronic Dipole "
cuadrupolo electronico "Electronic Quadrupole"
magnitud del dipolo "Dipole magnitude"
matriz de polarizabilidad " Symmetrized Local Polarizability Tensor "
#polarizabilidad isotropica "Isotropic Polarizability"

Dentro de "The Molecular Multipole Moments "

multipolos moleculares 
#tensor de polar. molecular "  Molecular Polarizability Tensor"


In [24]:
def leer_archivo(path_to_log):
    molec_dip_magnitude = [] #
    molec_dip_vector = [0,0,0] #
    molec_pol_tensor = [] #
    
    atoms_ = [] #
    bonds_ = [] #
    atoms_charge_mull = [] #
    atoms_charge_loprop = [] #
    atoms_dip_magnitude = [] #
    atoms_dip_vector = [] #
    atoms_iso_pol = []  #
    bonds_iso_pol = []  #
    
    with open(path_to_log, 'r') as f:
        molecular_prop = False
        atomic_domain = True
        mol_pol_tensor= False
        mulliken = False
        linea_ref_1=0
        linea_ref_2=0
        linea_ref_3=0
        atoms_dipoles = False
        
        for l_n, line in enumerate(f): #ITERO SOBRE LAS LINEAS DEL ARCHIVO
            line.replace(","," ")
            
#  ENERGIA
            if 'Total KS-DFT energy' in line:
                energy = float(line.split()[-1])
            
#  PROPIEDADES MOLECULARES
            if 'Molecular properties' in line:
                molecular_prop = True
                linea_ref_1 = l_n #GUARDO LA LINEA COMO REFERENCIA
                
##  DIPOLO MOLECULAR (VECTOR Y MAGNITUD)
            if l_n == linea_ref_1 + 7 and molecular_prop == True: # 7 LINEAS MAS ABAJO ESTAN LOS VALORES
                molec_dip_vector[0] = float(line.split()[-7])
                molec_dip_vector[1] = float(line.split()[-5])
                molec_dip_vector[2] = float(line.split()[-3])
                molec_dip_magnitude = float(line.split()[-1])

# PROPIEDADES LOCALIZADAS                
            if 'ATOMIC DOMAIN' in line:
                atoms_.append(line.split()[-1])
                atomic_domain = True
                bond_domain = False
            if 'BOND DOMAIN' in line:
                bonds_.append([line.split()[-2],line.split()[-1]])
                bond_domain = True
                atomic_domain = False
            
# ATOMIC DIPOLE VECTORS
            if 'Electronic Dipole' in line:
                atoms_dipoles = True
                linea_ref_2 = l_n
            if l_n == linea_ref_2 + 3 and atoms_dipoles == True and atomic_domain== True:
                vector = [float(i) for i in line.split()]
                atoms_dip_vector.append(vector)                

# DIPOLE MAGNITUDE
            if 'Dipole magnitude' in line and atoms_dipoles == True and atomic_domain== True:
                atoms_dip_magnitude.append(float(line.split()[-1]))
                
##CARGAS MULLIKEN 
            if 'Mulliken' in line:
                mulliken = True
            if mulliken == True and "N-E" in line:
                atoms_charge_mull = [float(i) for i in line.split()[1:]]
                
  #          if len(atoms_charge_mull) < #cantidad de atomos
  #              atoms_charge_mull.append([float(i) for i in line.split()[1:]])
  #              mulliken = False
            
##CARGA LOPROP POR ATOMO
            if 'Total charge' in line:
                charge = float(line.split()[-1])
                if atomic_domain == True:
                    atoms_charge_loprop.append(charge)
                    
##POLARIZABILIDAD POR DOMINIO            
            if 'Isotropic' in line:
                polarizability = float(line.split()[-1])
                if atomic_domain == True:
                    atoms_iso_pol.append(polarizability)
                elif atomic_domain == False:
                    bonds_iso_pol.append(polarizability)
                    
            if 'Molecular Polarizability Tensor' in line:
                mol_pol_tensor = True
                linea_ref_3=l_n
            if linea_ref_3+2<l_n and l_n<linea_ref_3 + 6 and mol_pol_tensor == True:
                molec_pol_tensor.append([float(i) for i in line.split()])
    f.close()
    
    
    total_iso_pol=[0]*len(atoms_)
    for i, atom in enumerate(atoms_):
        total_iso_pol[i] = atoms_iso_pol[i]
        for j, bond in enumerate(bonds_):
            if atom in bond:
                total_iso_pol[i] = total_iso_pol[i] + bonds_iso_pol[j]*0.5
    print('Dipolo molecular', molec_dip_magnitude)
    print('Dipolo molecular vector', molec_dip_vector)
    print('Tensor pol molecular', molec_pol_tensor)
    print('atomos', len(atoms_))
    print('enlaces', len(bonds_))
    print('cargas mulliken', atoms_charge_mull)
    print('cargas loprop', len(atoms_charge_loprop))
    print('dipolos atomicos', len(atoms_dip_magnitude))
    print('dipolos atomicos vector', len(atoms_dip_vector))
    print('pol atomica iso', len(atoms_iso_pol))
    print(' pol enlaces iso', len(bonds_iso_pol))
        
    return 

In [25]:
leer_archivo("C3H7N1O3_0.log")

Dipolo molecular 4.0125
Dipolo molecular vector [3.1368, -2.0719, 1.403]
Tensor pol molecular [[66.48717167459485], [1.8398975754023217, 65.7642780764651], [0.8333040243087171, -1.3082724970714683, 35.51106751570654]]
atomos 14
enlaces 14
cargas mulliken [-0.469, -0.5119]
cargas loprop 14
dipolos atomicos 14
dipolos atomicos vector 14
pol atomica iso 14
 pol enlaces iso 14


In [24]:
def extraer_pols(path_to_log):
    atoms_ = []
    bonds_ = []
    atoms_pol = []
    bonds_pol = []
    with open(path_to_log, 'r') as f:
        atomic_domain = True
        mol_tensor= False
        linea_ref=0
        for l_n, line in enumerate(f):
            line.replace(","," ")
            if 'ATOMIC DOMAIN' in line:
                atoms_.append(line.split()[-1])
                atomic_domain = True
            if 'BOND DOMAIN' in line:
                bonds_.append([line.split()[-2],line.split()[-1]])
                atomic_domain = False
            if 'Isotropic' in line:
                polarizability = line.split()[-1]
                if atomic_domain == True:
                    atoms_pol.append(polarizability)
                elif atomic_domain == False:
                    bonds_pol.append(polarizability)
            if 'Molecular Polarizability Tensor' in line:
                mol_pol_tensor = True
            if mol_tensor == True:
                linea_ref=linea_ref+1
                if 3<linea_ref and linea_ref<7:
                    values= line.split()
                    for value in values:
                        tensor.append(float(value))
    f.close()
    total_pol=[0]*len(atoms_)
    for i, atom in enumerate(atoms_):
        total_pol[i] = float(atoms_pol[i])
        for j, bond in enumerate(bonds_):
            if atom in bond:
                total_pol[i] = total_pol[i] + float(bonds_pol[j])*0.5
    return atoms_, total_pol, tensor

In [25]:
extraer_pols("water.log")

(['O1', 'H2', 'H3'], [3.02664763, 1.7124623200000002, 1.712703935])

In [18]:
print(atoms_)
print(atoms_pol)
print(bonds_)
print(bonds_pol)

['O1', 'H2', 'H3']
['3.02664763', '0.46483343', '0.46488007']
[['H2', 'O1'], ['H3', 'O1']]
['2.49525778', '2.49564773']


In [19]:
def corregir_pols(atoms_,atoms_pol,bonds_,bonds_pol):
    total_pol=[0]*len(atoms_)
    for i, atom in enumerate(atoms_):
        total_pol[i] = float(atoms_pol[i])
        for j, bond in enumerate(bonds_):
            if atom in bond:
                total_pol[i] = total_pol[i] + float(bonds_pol[j])*0.5
    return total_pol

In [20]:
corregir_pols(atoms_,atoms_pol,bonds_,bonds_pol)

[5.522100385, 1.7124623200000002, 1.712703935]